### 二値分類モデル　改訂版

<a href="https://colab.research.google.com/github/makaishi2/math_dl_book_info/blob/master/sample-notebook/ch08_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 必要ライブラリの導入

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib -qq

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.2f}'.format

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

### データ準備

#### データ読み込み

In [ ]:
from sklearn.datasets import load_iris

# アイリスデータセットをロード
iris = load_iris(as_frame=True)
#print(iris.DESCR)  # データセットの説明

df = iris.data
df.columns = ['がく片長', 'がく片幅', '花弁長', '花弁幅']
display(df.head())

#### データ絞り込み

In [ ]:
# データ絞り込み
#   クラス0, 1のみ
#   項目がく片長とがく片幅のみ
x_data = df[['がく片長','がく片幅']].head(100)
y_data = iris.target[:100].values

print('x_data')
print(f'shape: {x_data.shape}')
print(f'先頭5行:\n{x_data[:5]}\n')

print('y_data')
print(f'shape: {y_data.shape}')
print(f'先頭5行:\n{y_data[:5]}')

#### ダミー変数追加

In [ ]:
x_data2 = np.insert(x_data, 0, 1.0, axis=1)

print('ダミー変数追加後')
print(f'shape: {x_data2.shape}')
print(f'先頭5行:\n{x_data2[:5]}\n')


#### 訓練・テストデータ分割

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_data2, y_data, train_size=70, test_size=30,
    random_state=123)

print(f'x_train.shape: {x_train.shape}')
print(f'x_test.shaoe: {x_test.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'y_test.shape: {y_test.shape}')

 #### 学習データの散布図表示

#### y_trainの値でx_trainを分割

In [ ]:
x_t0 = x_train[y_train == 0]
x_t1 = x_train[y_train == 1]

#### 散布図表示

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 散布図表示
plt.scatter(x_t0[:,1], x_t0[:,2], marker='x', c='b', label='0 (setosa)')
plt.scatter(x_t1[:,1], x_t1[:,2], marker='o', c='k', label='1 (versicolor)')

# グラフのキレイ化
plt.title('訓練データの散布図')
plt.xlabel('がく片長')
plt.ylabel('がく片幅')
plt.legend()
plt.grid()
plt.show()

### 学習準備

#### 学習用変数の設定

In [ ]:
# 学習用変数の設定
x = x_train
yt = y_train

# 入力データ x の先頭5行の表示 (ダミーデータを含む)
print(x[:5])

# 正解値 yt の先頭5行の表示
print(yt[:5])

### 関数定義

#### シグモイド関数

In [ ]:
def sigmoid(x):
    return 1/(1+ np.exp(-x))

#### 予測関数

In [ ]:
def pred(x, w):
    return sigmoid(x @ w)

#### 損失関数(交差エントロピー関数)

In [ ]:
def cross_entropy(yt, yp):
    # 交差エントロピーの計算(この段階ではベクトル)
    ce1 = -(yt * np.log(yp) + (1 - yt) * np.log(1 - yp))
    # 交差エントロピーベクトルの平均値を計算
    return(np.mean(ce1))

#### classify (確率値から0/1を判定)

In [ ]:
def classify(y):
    return np.where(y < 0.5, 0, 1)

 ####  evaluate(損失と精度を計算)

In [ ]:
from sklearn.metrics import accuracy_score
def evaluate(xt, yt, w):

    # 予測値の計算
    yp = pred(xt, w)

    # 損失関数値の計算
    loss = cross_entropy(yt, yp)

    # 予測値(確率値)を0または1に変換
    yp_b = classify(yp)

    #精度の算出
    score = accuracy_score(yt, yp_b)
    return loss, score

### 学習

#### 初期設定

In [ ]:
# 標本数
M  = x.shape[0]
# 入力次元数(ダミー変数を含む)
D = x.shape[1]

# 繰り返し回数
iters = 5000

# 学習率
alpha = 0.01

# 初期値
w = np.ones(D)

# 評価結果記録用 (損失関数と精度)
history = np.zeros((0,3))

 #### 繰り返し処理

In [ ]:
for k in range(iters):

    # 予測値の計算
    yp = pred(x, w)

    # 誤差の計算
    yd = yp - yt

    # 勾配降下法の実施
    w = w - alpha * (x.T @ yd) / M

    # ログ記録用
    if ( k % 10 == 0):
        loss, score = evaluate(x_test, y_test, w)
        history = np.vstack((history,
            np.array([k, loss, score])))
        print( f'iter = {k}  loss = {loss:.04f} score = {score:.04f}')


### 結果分析

#### 損失・精度確認

In [ ]:
print(f'初期状態: 損失関数:{history[0,1]:.04f} 精度:{history[0,2]:.04f}')
print(f'最終状態: 損失関数:{history[-1,1]:.04f} 精度:{history[-1,2]:.04f}')

#### 学習曲線表示(損失)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,1])

# グラフのキレイ化
plt.title('学習曲線(損失)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 学習曲線表示(精度)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,2])

# グラフのキレイ化
plt.title('学習曲線(精度)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 決定境界表示

In [ ]:
# y_testの値を元にx_testを分割
x_t0 = x_test[y_test==0]
x_t1 = x_test[y_test==1]

# 決定境界描画用 x(x1)の値からy(x2)の値を計算する
def b(x, w):
    return(-(w[0] + w[1] * x)/ w[2])

# 散布図のx1の最小値と最大値
xl = np.asarray([x[:,1].min(), x[:,1].max()])

# 対応する決定境界のy座標を計算
yl = b(xl, w)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 散布図の表示
plt.scatter(x_t0[:,1], x_t0[:,2], marker='x',c='b', s=50, label='0 (setosa)')
plt.scatter(x_t1[:,1], x_t1[:,2], marker='o',c='k', s=50, label='1 (versicolor)')

# 決定境界の直線を追記
plt.plot(xl, yl, c='b')

# グラフのキレイ化
plt.title('テストデータの散布図と決定境界')
plt.xlabel('がく片長')
plt.ylabel('がく片幅')
plt.legend()
plt.grid()
plt.show()

#### 予測関数の3次元曲面表示

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
x1 = np.linspace(4, 7.5, 100)
x2 = np.linspace(2, 4.5, 100)
xx1, xx2 = np.meshgrid(x1, x2)
xxx = np.asarray([np.ones(xx1.ravel().shape),
    xx1.ravel(), xx2.ravel()]).T
c = pred(xxx, w).reshape(xx1.shape)
plt.figure(figsize=(8,8))
ax = plt.subplot(1, 1, 1, projection='3d')
ax.plot_surface(xx1, xx2, c, color='blue',
    edgecolor='black', rstride=10, cstride=10, alpha=0.1)
ax.scatter(x_t1[:,1], x_t1[:,2], 1, s=20, alpha=0.9, marker='o', c='b')
ax.scatter(x_t0[:,1], x_t0[:,2], 0, s=20, alpha=0.9, marker='s', c='b')
ax.set_xlim(4,7.5)
ax.set_ylim(2,4.5)
ax.view_init(elev=20, azim=60)

### scikit-learnを利用したモデルとの比較

In [ ]:
# 必要ライブラリのロード
from sklearn.linear_model import LogisticRegression
from sklearn import svm

# モデル生成
model_lr = LogisticRegression(solver='liblinear')
model_svm = svm.SVC(kernel='linear')

# 機械学習実施
model_lr.fit(x, yt)
model_svm.fit(x, yt)

#### 決定直線の算出

In [ ]:
# 線形回帰
# 切片の値
lr_w0 = model_lr.intercept_[0]
# x1(sepal_length)の係数
lr_w1 = model_lr.coef_[0,1]
# x2(sepal_width)の係数
lr_w2 = model_lr.coef_[0,2]

def rl(x):
    wk = lr_w0 + lr_w1 * x
    wk2 = -wk / lr_w2
    return(wk2)

y_rl = rl(xl)

# 結果確認
print(xl, y_rl)

In [ ]:
# SVM
# 切片の値
svm_w0 = model_svm.intercept_[0]
# x1(sepal_length)の係数
svm_w1 = model_svm.coef_[0,1]
# x2(sepal_width)の係数
svm_w2 = model_svm.coef_[0,2]

def svm(x):
    wk = svm_w0 + svm_w1 * x
    wk2 = -wk / svm_w2
    return(wk2)

y_svm = svm(xl)

# 結果確認
print(xl,y_svm)

#### 散布図と決定直線の同時表示

In [ ]:
plt.figure(figsize=(6,6))

# 散布図の表示
plt.scatter(x_t0[:,1], x_t0[:,2], marker='x',c='b', s=50, label='0 (setosa)')
plt.scatter(x_t1[:,1], x_t1[:,2], marker='o',c='k', s=50, label='1 (versicolor)')

# 決定直線の表示
plt.plot(xl, yl, linewidth=2, c='k', label='Hands On')
# lr model
plt.plot(xl, y_rl, linewidth=2, c='k', linestyle="--", label='scikit LR')
# svm
plt.plot(xl, y_svm, linewidth=2, c='b', linestyle="-.", label='scikit SVM')

# グラフのキレイ化
plt.title('テストデータの散布図と決定境界')
plt.xlabel('がく片長')
plt.ylabel('がく片幅')
plt.legend()
plt.grid()
plt.show()

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions